In [1]:
import json
from os.path import join, dirname
import watson_developer_cloud
from watson_developer_cloud import SpeechToTextV1
import watson_developer_cloud.natural_language_understanding.features.v1 as features
import pandas as pd
import nltk

speech_to_text = SpeechToTextV1(
    username='6d736cd4-1e01-4d6c-9c61-78ce7f803024',
    password='SBE03B3o5PTP',
    x_watson_learning_opt_out=False
)

nlp_understand = watson_developer_cloud.NaturalLanguageUnderstandingV1(version='2017-02-27',
                                                            username='991ee8cd-25f9-4b5a-a1f8-cfe5834dfcb9',
                                                            password='Jtera8xCDVup')




In [2]:
#print(json.dumps(speech_to_text.models(), indent=2))

#print(json.dumps(speech_to_text.get_model('en-US_BroadbandModel'), indent=2))

with open('./chicken_dinner.mp3','rb') as audio_file:
    txt = json.dumps(speech_to_text.recognize(
        audio_file, content_type='audio/mp3', timestamps=True,
        word_confidence=True),
        indent=2)

In [3]:
json_text = json.loads(txt)

In [4]:
json_text

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.489,
     'timestamps': [['I', 1.0, 1.18],
      ['ate', 1.18, 1.69],
      ['chicken', 1.86, 2.3],
      ['for', 2.3, 2.53],
      ['defeat', 2.53, 2.95]],
     'transcript': 'I ate chicken for defeat ',
     'word_confidence': [['I', 0.772],
      ['ate', 0.827],
      ['chicken', 0.134],
      ['for', 0.62],
      ['defeat', 0.258]]}],
   'final': True}],
 'warnings': ['Unknown arguments: continuous.']}

In [5]:
text = json_text['results'][0]['alternatives'][0]['transcript']

In [6]:
text

'I ate chicken for defeat '

In [7]:
keywords = nlp_understand.analyze(text=text, features=[features.Keywords()])

In [8]:
keywords

{'keywords': [{'relevance': 0.945436, 'text': 'defeat'},
  {'relevance': 0.867494, 'text': 'chicken'}],
 'language': 'en',
 'usage': {'features': 1, 'text_characters': 25, 'text_units': 1}}

In [9]:
s = pd.Series(keywords)

In [10]:
keyword_list = []
for k in s.keywords:
    keyword_list.append(k['text'])

In [11]:
keyword_list.append('hamburger')

In [12]:
from nltk.corpus import wordnet as wn
food = wn.synset('food.n.02')
nltk_food = list(set([w for s in food.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))

In [13]:
foods = [k for k in keyword_list if k in nltk_food]

In [14]:
foods

['chicken', 'hamburger']

In [15]:
foods.append('beef')

In [16]:
usda = pd.read_csv('usda_sample-2015.csv')

In [17]:
usda = usda.drop(['brand_name', 'brand_id', 
                'item_id', 'upc','item_description','item_type','nf_ingredient_statement','updated_at'], axis=1)

In [18]:
usda.columns

Index(['item_name', 'nf_calories', 'nf_calories_from_fat', 'nf_total_fat',
       'nf_saturated_fat', 'nf_trans_fatty_acid', 'nf_polyunsaturated_fat',
       'nf_monounsaturated_fat', 'nf_cholesterol', 'nf_sodium',
       'nf_total_carbohydrate', 'nf_dietary_fiber', 'nf_sugars', 'nf_protein',
       'nf_vitamin_a_dv', 'nf_vitamin_c_dv', 'nf_calcium_dv', 'nf_iron_dv',
       'nf_servings_per_container', 'nf_serving_size_qty',
       'nf_serving_size_unit', 'nf_serving_weight_grams'],
      dtype='object')

In [47]:
usda.head()

,item_name,nf_calories,nf_calories_from_fat,nf_total_fat,nf_saturated_fat,nf_trans_fatty_acid,nf_polyunsaturated_fat,nf_monounsaturated_fat,nf_cholesterol,nf_sodium,...,nf_sugars,nf_protein,nf_vitamin_a_dv,nf_vitamin_c_dv,nf_calcium_dv,nf_iron_dv,nf_servings_per_container,nf_serving_size_qty,nf_serving_size_unit,nf_serving_weight_grams
0,"Veal, variety meats and by-products, tongue, c...",171,77,8,3,null,0,3,202,54,...,0,21,0,8,0,9,null,3,oz,85
1,"Babyfood, cereal, barley, dry - 0.5 oz",56,8,0,0,null,0,0,0,1,...,1,1,0,0,11,39,null,0,oz,15
2,"Broadbeans (fava beans), mature seeds, raw - 1...",511,20,2,0,null,0,0,0,19,...,8,39,1,3,15,55,null,1,cup,150
3,"Beef, chuck eye steak, boneless, separable lea...",856,565,62,26,null,4,29,261,253,...,0,72,1,0,5,44,null,1,steak,384
4,"Cheese, cottage, creamed, large or small curd ...",205,81,9,3,null,0,1,35,764,...,5,23,5,0,17,0,null,1,"cup, large curd (not packed)",210


In [32]:
usda.iloc[10]

item_name                    Vinegar, red wine - 1 cup
nf_calories                                         45
nf_calories_from_fat                                 0
nf_total_fat                                         0
nf_saturated_fat                                     0
nf_trans_fatty_acid                               null
nf_polyunsaturated_fat                               0
nf_monounsaturated_fat                               0
nf_cholesterol                                       0
nf_sodium                                           19
nf_total_carbohydrate                                0
nf_dietary_fiber                                     0
nf_sugars                                            0
nf_protein                                           0
nf_vitamin_a_dv                                      0
nf_vitamin_c_dv                                      1
nf_calcium_dv                                        1
nf_iron_dv                                           5
nf_serving

In [33]:
pcf = usda[['item_name','nf_total_fat','nf_total_carbohydrate','nf_protein']]

In [34]:
pcf['total'] = pcf['nf_total_fat'] + pcf['nf_total_carbohydrate'] + pcf['nf_protein']

/home/anilbey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
pcf['fat_pct'] = pcf.nf_total_fat / pcf.total

/home/anilbey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
pcf['protein_pct'] = pcf.nf_protein / pcf. total

/home/anilbey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
pcf['carbon_pct'] = pcf.nf_total_carbohydrate / pcf. total

/home/anilbey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
pcf = pcf.round({'fat_pct': 2, 'protein_pct': 2, 'carbon_pct': 2})

In [45]:
pcf = pcf.fillna(0)

In [46]:
pcf

,item_name,nf_total_fat,nf_total_carbohydrate,nf_protein,total,fat_pct,protein_pct,carbon_pct
0,"Veal, variety meats and by-products, tongue, c...",8,0,21,29,0.28,0.72,0.00
1,"Babyfood, cereal, barley, dry - 0.5 oz",0,10,1,11,0.00,0.09,0.91
2,"Broadbeans (fava beans), mature seeds, raw - 1...",2,87,39,128,0.02,0.30,0.68
3,"Beef, chuck eye steak, boneless, separable lea...",62,0,72,134,0.46,0.54,0.00
4,"Cheese, cottage, creamed, large or small curd ...",9,7,23,39,0.23,0.59,0.18
5,"PREGO Pasta, Chunky Garden Mushroom and Green ...",3,13,2,18,0.17,0.11,0.72
6,"McDONALD'S, Apple Dippers with Low Fat Caramel...",0,23,0,23,0.00,0.00,1.00
7,"CAMPBELL'S CHUNKY Soups, Sizzlin' Steak- Grill...",2,26,15,43,0.05,0.35,0.60
8,"Chicken, broilers or fryers, drumstick, meat o...",3,0,12,15,0.20,0.80,0.00
9,"Babyfood, oatmeal cereal with fruit, dry, inst...",0,3,0,3,0.00,0.00,1.00


In [19]:
matched_food = []
for f in foods:
    for index, row in usda.iterrows():
        if f in row['item_name']:
            matched_food.append(index)
            #returns the first occurance of the food, e.g. chicken
            break

In [20]:
selected_nutrition_df = usda.iloc[matched_food,:]

In [21]:
selected_nutrition_df

,item_name,nf_calories,nf_calories_from_fat,nf_total_fat,nf_saturated_fat,nf_trans_fatty_acid,nf_polyunsaturated_fat,nf_monounsaturated_fat,nf_cholesterol,nf_sodium,...,nf_sugars,nf_protein,nf_vitamin_a_dv,nf_vitamin_c_dv,nf_calcium_dv,nf_iron_dv,nf_servings_per_container,nf_serving_size_qty,nf_serving_size_unit,nf_serving_weight_grams
49,"Chicken, broilers or fryers, giblets, raw - 1 ...",28,9,1,0,null,0,0,60,17,...,0,4,40,6,0,7,null,1,unit (yield from 1 lb ready-to-cook chicken),23
21,"Frankfurter, beef, pork, and turkey, fat free ...",62,8,0,0,null,0,0,23,501,...,0,7,0,22,3,5,null,1,frank 1 NLEA serving,57
